In [1]:
from transformers import RobertaForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoModelForMaskedLM, AutoModelForCausalLM
import torch, datasets, sacremoses

device = ("cuda" if torch.cuda.is_available() else "cpu")

model_name = "flax-community/papuGaPT2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.config.pad_token_id = tokenizer.eos_token_id

## Data


In [2]:
def group_texts(examples, block_size=512):
    
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = (len(concatenated["input_ids"]) // block_size) * block_size

    result = {}
    for k, v in concatenated.items():
        result[k] = [v[i:i + block_size] for i in range(0, total_length, block_size)]
        
    return result

In [3]:
ds = datasets.load_dataset("text", data_files={
   "train": "pan_tadeusz_1_10.txt",
   "validation": "pan_tadeusz_11.txt",
   "test": "pan_tadeusz_12.txt",
})


def tokenize_function(examples):
   return tokenizer(examples["text"])


tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_datasets = tokenized_datasets.map(group_texts, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [4]:
import torch
import torch.nn.functional as F

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)
    
    shift_logits = logits[..., :-1, :].reshape(-1, logits.shape[-1])
    shift_labels = labels[..., 1:].reshape(-1)
    loss = F.cross_entropy(shift_logits, shift_labels)
    perplexity = torch.exp(loss).item()
    return {"perplexity": perplexity}

## Base check

In [5]:
model.eval()
prompt = "Jam jest Jacek"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=200,
        do_sample=True,
        temperature=.9,
        top_p=0.9
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text before training:\n", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated text before training:
 Jam jest Jacek, ale ten jest po to, żeby służyć w tym świętym miejscu. On jest dla ciebie i dla ciebie. Ja jestem, a ty jesteś z tego, a więc twoja dusza jest ze mną, czy to jesteś Ty, czy to jest twoja dusza i twoja dusza. Ja jestem twoim Ojcem, i ty jesteś ze mną, a więc twoja dusza i twoja dusza jest ze mną. Ja jestem twoim Ojcem i ty jesteś ze mną. Ja jestem twój Ojciec, i ty jesteś moim Ojcem. To jest to, co robię, w jaki sposób robię, to, co mówię, to, co robię. Ja jestem twoim Ojcem i ty jesteś moim Ojcem, to, co robię, co mówię, to, co robię, to, co mówię. Ja jestem twoim Ojcem, a ty jesteś moim Ojcem, a więc twój Ojciec jest twoim Ojcem, ja jestem twoim Ojcem. Ja jestem twoim Ojcem i ty jesteś moim Ojcem. To jest to, co


## Training

In [6]:
from transformers import TrainerCallback

class GenerateTextCallback(TrainerCallback):
    def __init__(self, tokenizer, prompt="Jam jest Jacek", max_length=200):
        self.tokenizer = tokenizer
        self.prompt = prompt
        self.max_length = max_length

    def on_epoch_end(self, args, state, control, **kwargs):
        model = kwargs['model']
        model.eval()

        input_ids = self.tokenizer(self.prompt, return_tensors="pt").input_ids.to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_length=self.max_length,
                do_sample=True,
                top_p=0.9,
                temperature=.9,
                num_return_sequences=1
            ) 
        
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n--- Generated text after epoch {state.epoch}:")
        print(generated_text)
        print("---------------------------\n")

In [7]:
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./decoder-pan-tadeusz",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="perplexity",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[GenerateTextCallback(tokenizer),  EarlyStoppingCallback(early_stopping_patience=2)]
)


trainer.train()

C:\Users\litwi\AppData\Local\Temp\ipykernel_31952\1012462473.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss,Perplexity
1,No log,4.834122,125.726967
2,No log,4.761708,116.944138
3,4.696200,4.731475,113.461487
4,4.696200,4.717376,111.873055
5,4.408800,4.713063,111.391479


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Generated text after epoch 1.0:
Jam jest Jacek, że go tu przyprowadziłeś i za kim on stał, że za nim z Tobą stał, że za nim wołał i wołał? Bo ja jestem Jacek, ja mu, Jacku, mówię, że cię szukam, ale on nie może mnie znaleźć; ja go nie znajdę; on szuka mnie; ja nie znajdę, ja go nie znajdę; ja go nie znajdę; ja nie znajdę, ja go nie znajdę; ja go nie znajdę; ja go nie znajdę; ja go nie znajdę, ja go nie znajdę; ja go nie znajdę, ja go nie znajdę; ja go nie znajdę; ja go nie znajdę, ja go nie znajdę; ja go nie znajdę; ja go nie znajdę; ja go nie znajdę; ja go nie znajdę, ja go nie znajdę; ja go nie znajdę, ja go nie znajdę; ja go nie znajdę, ja go nie znajdę; ja go nie znajdę, ja go nie znajdę; ja go nie znajdę; ja go nie znajdę, ja
---------------------------



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


--- Generated text after epoch 4.0:
Jam jest Jacek, niechże się Jacek zająknie i jak człowiek z daleka, niech spojrzy na niego; lecz kto go nie zobaczy, ten nie będzie jaśniał z jego oczu, jako jaśniejący Janek z dalekich krajów».    «Jeruzalem! niech jaśnieje w całym twoim świecie!». «Panie Boże! Panie Boże! Boże, jaśniejesz! niech jaśniejesz! niech jaśniejesz!» «Och, niechaj jaśniejesz!» «Panie Boże! niechaj jaśniejesz!» «Oj, niechaj jaśniejesz!» «O! niechaj jaśniejesz!» «Oj, niechaj jaśniejesz!». «Niechaj jaśniejesz!» «Oj, niechaj jaśniejesz!» «Panie Boże! jaśniejesz!». «Niechaj Jaśniejesz!» «Panie Boże! Boże, jaśniejesz!» «Panie Boże, jaśniejesz!» «Nie
---------------------------



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Generated text after epoch 5.0:
Jam jest Jacek, to znaczy ja, to znaczy Jacek, jak ja mówię: «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!» «Ja, Jacek!». Jacek ma trzynaście lat i w tym wieku jest już człowiekiem dojrzałym, bardzo gotowym do służby Bożej.Wziął, co mu się podobało, wjechał do miasta, gdzie go w domu zastali.Wziął do roboty. W tej robocie widział, że w mieście nie było dla niego miejsca.W mieście nie było miejsca, gdzie byłby panować nad tymi ludźmi.To są ludzie młodzi, a młodzież w mieście jest bardzo duża.Jak pan widzi, pan był we wsi bardzo dobrze zżyty i w tej wsi, więc pan ma się stać na miejscu moim
---------------------------



There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=120, training_loss=4.516413879394531, metrics={'train_runtime': 447.6973, 'train_samples_per_second': 2.122, 'train_steps_per_second': 0.268, 'total_flos': 248227430400000.0, 'train_loss': 4.516413879394531, 'epoch': 5.0})